<h1>Locate Exchange Rate Anomolies</h1>
<h2>Example 1 - Simplest Approach</h2>
This is an extremely simple example. In this approach, we just look at the closing price at midnight of each day. If the price difference is greater than a threshold that we set then we flag it as an anomoly.

Start by pulling in the requisite libraries.

In [1]:
import sys
import csv
import datetime
import mysql.connector

Now connect to the database. This code should be the same for most programs that do analysis of the data.

In [2]:
try:
    cnx = mysql.connector.connect(user='ec2-user', database='plato', host='localhost')
    cursor = cnx.cursor()
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Problem with user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
    sys.exit()


To configure the search, set the Ticker to the desired value, and set the Threshold to trigger it as an anomoly:

In [5]:
Ticker = "AUDUSD"
Threshold = float(0.02)  # that is, 2%

For this example, we only need to examine the Exch table in the database. It is defined as follows:

<pre>
CREATE TABLE Exch (
    XID BIGINT NOT NULL AUTO_INCREMENT,                     -- unique id for this record
    Dt DATETIME NOT NULL DEFAULT '1970-01-01 00:00:00',     -- point in time when these values are valid
    Ticker VARCHAR(10) NOT NULL DEFAULT '',                 -- the two currencies
    Open DECIMAL(19,4) NOT NULL DEFAULT 0,                  -- Opening value for this minute
    High DECIMAL(19,4) NOT NULL DEFAULT 0,                  -- High value during this minute
    Low DECIMAL(19,4) NOT NULL DEFAULT 0,                   -- Low value during this minute
    Close DECIMAL(19,4) NOT NULL DEFAULT 0,                 -- Closing value for this minute
    LastModTime TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    LastModBy BIGINT NOT NULL DEFAULT 0,                    -- UID
    CreateTime TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    CreateBy BIGINT NOT NULL DEFAULT 0,                     -- UID
    CONSTRAINT Alpha UNIQUE(Dt,Ticker),                     -- no more than one of these
    PRIMARY KEY(XID)
);
</pre>

In the code, we only need the closing rate (Close) and the date (Dt) for every database Row at midnight.

In [8]:
query = 'SELECT Dt,Close FROM Exch WHERE Ticker="{}" AND MINUTE(Dt)=0 AND HOUR(Dt)=0'.format(Ticker)
cursor.execute(query)
rows = cursor.fetchall()
l = len(rows) - 1

In this case we just put all the matches into an array called <code>rows</code> then loop over it. Each value in the array is another array consisting of <code>Dt</code> at index 0 and <code>Close</code> at index 1.

In [10]:
hits = 0    # number of anomlies that matched our criteria
i = 0       # counter
while i < l:
    v1 = rows[i][1]     # Closing value on dt
    v2 = rows[i+1][1]   # Closing value on dt + 1day
    dt = rows[i][0]     # datetime of this record (midnight each day)
    delta = abs(v2-v1)  # difference between this record and the next record's closing exch rate
    thresh = float(v1) * Threshold; # threshold amount
    if  delta > thresh:
        d = dt.strftime("%b %d, %Y")
        hits = hits + 1
        print('{}\t{} v1={}  v2={} delta={}, Threshold={:.4f}'.format(hits,d,v1,v2,delta,thresh))
    i = i+1
print(f"total anomolies found: {hits}")


1	Mar 15, 2011 v1=1.0100  v2=0.9890 delta=0.0210, Threshold=0.0202
2	Aug 04, 2011 v1=1.0771  v2=1.0448 delta=0.0323, Threshold=0.0215
3	Aug 08, 2011 v1=1.0412  v2=1.0187 delta=0.0225, Threshold=0.0208
4	Aug 10, 2011 v1=1.0389  v2=1.0151 delta=0.0238, Threshold=0.0208
5	Sep 21, 2011 v1=1.0264  v2=1.0050 delta=0.0214, Threshold=0.0205
6	Sep 22, 2011 v1=1.0050  v2=0.9744 delta=0.0306, Threshold=0.0201
7	Oct 10, 2011 v1=0.9758  v2=0.9994 delta=0.0236, Threshold=0.0195
8	Oct 27, 2011 v1=1.0395  v2=1.0712 delta=0.0317, Threshold=0.0208
9	Nov 01, 2011 v1=1.0537  v2=1.0292 delta=0.0245, Threshold=0.0211
10	Nov 09, 2011 v1=1.0379  v2=1.0117 delta=0.0262, Threshold=0.0208
11	Nov 30, 2011 v1=1.0036  v2=1.0267 delta=0.0231, Threshold=0.0201
12	Jun 29, 2012 v1=1.0039  v2=1.0258 delta=0.0219, Threshold=0.0201
13	Jun 19, 2013 v1=0.9483  v2=0.9279 delta=0.0204, Threshold=0.0190
14	Nov 05, 2014 v1=0.8752  v2=0.8571 delta=0.0181, Threshold=0.0175
15	Apr 28, 2015 v1=0.7847  v2=0.8021 delta=0.0174, Thresh

While this is simple to code, note that if the value of the exchange rate can change dramatically at any point after midnight but returned to a price within the threshold before midnight the next day this approach will not flag it.